In [1]:
import pandas as pd
import os
import sys
from math import floor
import pylev
import re
import numpy as np

In [2]:
curr_dir = os.path.abspath(os.path.dirname("__file__"))

In [3]:
def get_fp_val(row, values):
#     print(row)
#     return 0
    if (row.Fantasypros_rank >= len(values)):
        return values.iloc[-1]
    rd = int(floor(row.Fantasypros_rank))
    diff = row.Fantasypros_rank - rd
    top_val = values.iloc[rd - 1]
    bot_val = values.iloc[rd]
    return (1 - diff) * top_val + diff * bot_val

def get_bonuses(row, bonuses):
    pos = row['Position'].lower()
    rank = row['Position_rank']
    bonus = bonuses[pos + '12'].iloc[0] - bonuses[pos + '10'].iloc[0]
    return bonus

def get_pick_val_bs(row, df):
    if np.isnan(row.my_pick):
        return row.my_pick
    return df.Value.loc[row.name] - df.Value.iloc[int(min(row.my_pick - 1, len(df) - 1))]

def get_pick_val_fp(row, df):
    if np.isnan(row.my_pick):
        return row.my_pick
    return df.Value.iloc[row.Rank_fp - 1] - df.Value.iloc[int(min(row.my_pick - 1, len(df) - 1))]
    
def get_my_pick(e, buff=0):
    my_picks = [9, 16]
    for i in range(15):
        my_picks.append(my_picks[i] + 24)
    for i, m in enumerate(my_picks):
        if m > e-buff:
            return my_picks[max(i - 1, 0)]
    return my_picks[-1]
        
def round_target(p):
    if np.isnan(p):
        return p
    return int((p-1) / 12) + 1
        
def get_name(s):
    i = s.index('.')
    return s[i+2:]

def espn_rank(name, espn):
    names = espn.Name.tolist()
    name_match = min(names, key=lambda x: pylev.levenshtein(x.lower(), name.lower()))
    i = names.index(name_match)
    return i + 1

def get_pos_rank(pos):
    d = {}
    l = []
    for p in pos:
        if p not in d:
            d[p] = 1
        l.append(d[p])
        d[p] += 1
    return l

In [4]:
fp = pd.read_csv(os.path.join(curr_dir, "FantasyPros_2016_Draft_Overall_Rankings.csv"))

In [5]:
regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string
re.compile('[^a-zA-Z]').sub('', 'ab3d*E')
fp['Position_alpha'] = fp['Position '].apply(lambda s: re.compile('[^a-zA-Z]').sub('', s))
fp.to_csv('fantasypros.csv')

In [4]:
using_fp = ''
df = pd.read_csv(os.path.join(curr_dir, "fantasy_rankings.csv"))
espn = pd.read_csv(os.path.join(curr_dir, "espn.csv"))
espn['Name'] = espn['RANK, PLAYER'].apply(get_name)
df['espn_rank'] = df['Player' + using_fp].apply(lambda x: espn_rank(x, espn))
df['my_pick'] = df.espn_rank.apply(get_my_pick)
df['round_target'] = df['my_pick'].apply(round_target)
df['pos_rank'] = get_pos_rank(df['Pos'])
df['pos_rank_str'] = [p + str(r) for p, r in zip(df['Pos'], df.pos_rank)]
# df['diff'] = df.index - df.espn_rank + 1
df['pick_val_bs'] = df.apply(lambda x: get_pick_val_bs(x, df), axis=1)
df['pick_val_fp'] = df.apply(lambda x: get_pick_val_fp(x, df), axis=1)
df = pd.concat([df[df.Rank_fp <= 100], df[df.Rank_fp > 100].sort_values('Rank_fp')], ignore_index=True)
df.index = df.index + 1

In [6]:
df[['Player', 'Pos', 'pos_rank_str', 'Rank_avg_fp', 'espn_rank']].to_csv('print.csv')
# df

In [75]:
df[(df['Pos'] == 'QB') & (df.index<120)].sort_values('pick_val_bs', ascending=False)[['Player', 'Pos', 'espn_rank', 'my_pick', 'round_target']]

,Player,Pos,espn_rank,my_pick,round_target
29,Cam Newton,QB,35,33,3
104,Tyrod Taylor,QB,131,129,11
106,Derek Carr,QB,134,129,11
111,Kirk Cousins,QB,133,129,11
115,Tony Romo,QB,132,129,11
41,Aaron Rodgers,QB,49,40,4
74,Drew Brees,QB,68,64,6
67,Andrew Luck,QB,61,57,5
47,Russell Wilson,QB,50,40,4
96,Eli Manning,QB,88,88,8


In [ ]:
df = pd.read_csv(os.path.join(curr_dir, "fantasy_rankings.csv"))
bonuses = pd.read_csv(os.path.join(curr_dir, "bonuses.csv"))
espn = pd.read_csv(os.path.join(curr_dir, "espn.csv"))
df = df[pd.notnull(df.Value)].iloc[:100]

df['Fantasypros_value'] = df.apply(lambda x: get_fp_val(x, df.Value), axis=1)
df['Position_rank'] = df.Position.apply(lambda x: int(x[2:]))
df.Position = df.Position.apply(lambda x: x[:2])
df['Bonus_adj'] = df.apply(lambda x: get_bonuses(x, bonuses), axis=1) + df.Fantasypros_value
df = df.sort_values('Bonus_adj', ascending=False).reset_index(drop=True)
espn['Name'] = espn['RANK, PLAYER'].apply(get_name)
df['espn_rank'] = df['Fantasypros_name'].apply(lambda x: espn_rank(x, espn))

In [ ]:
def get_trade_values(df, picks):
    picks = [p - 1 for p in picks]
    return df.iloc[picks].Bonus_adj.mean()

In [ ]:
get_trade_values(df, [7, 18, 42,66, 90])

In [ ]:
get_trade_values(df, [9, 16, 40, 64, 88])
get_trade_values(df, [9, 16, 40, 64, 88])

In [ ]:
df.to_csv('my_ranks.csv')

In [ ]:
df[['Fantasypros_name', 'Fantasypros_rank', 'espn_rank']].iloc[40:]

In [ ]:
df.iloc[50:]